In [1]:
import pandas as pd
import os

In [2]:
grocery_sales = pd.read_csv(r'C:\Users\Azamat.ma\Desktop\ETL\main\datalab_export_2024-08-04 12_14_37.csv')

In [10]:
import pandas as pd
import os

# Start here...

def extract(sales_date, extra_data):
    extra_df = pd.read_parquet(extra_data)
    merged_df = sales_date.merge(extra_df, on='index')
    return merged_df

merged_df = extract(grocery_sales, "extra_data.parquet")


def transform(raw_data):
    raw_data.fillna(
    {
        'CPI':raw_data['CPI'].mean(),
        'Weekly_Sales': raw_data['Weekly_Sales'].mean(),
        'Unemployment': raw_data['Unemployment'].mean()
    }, inplace=True
    )
    
    raw_data['Date'] = pd.to_datetime(raw_data['Date'], errors='coerce')
    raw_data['Month'] = raw_data['Date'].dt.month  # Add this line to extract the month
    
    raw_data = raw_data.loc[raw_data["Weekly_Sales"] > 10000, :]
    
    raw_data = raw_data.drop(["index", "Temperature", "Fuel_Price", "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5", "Type", "Size", "Date"], axis = 1)
    return raw_data

clean_data = transform(merged_df)


def avg_monthly_sales(clean_data):
    holiday_sales = clean_data[['Month', 'Weekly_Sales']]
    
    holiday_sales = (holiday_sales.groupby('Month').agg(Avg_Sales = ('Weekly_Sales', 'mean')).reset_index().round(2))
    return holiday_sales

agg_data = avg_monthly_sales(clean_data)
print(agg_data)

def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    full_data.to_csv(full_data_file_path, index=False)
    agg_data.to_csv(agg_data_file_path, index=False)
    
load(clean_data, "clean_data.csv", agg_data, "agg_data.csv")


def validation(file_path):
  	# Use the "os" package to check whether a path exists
    file_exists = os.path.exists(file_path)
    # Raise an exception if the path doesn't exist, hence, if there is no file found on a given path
    if not file_exists:
        raise Exception(f"There is no file at the path {file_path}")

# Call the validation() function and pass first, the cleaned DataFrame path, and then the aggregated DataFrame path
validation("clean_data.csv")
validation("agg_data.csv")

    Month  Avg_Sales
0     1.0   40001.26
1     2.0   40932.18
2     3.0   39731.28
3     4.0   40262.77
4     5.0   40077.05
5     6.0   42214.58
6     7.0   40331.23
7     8.0   40031.06
8     9.0   40219.42
9    10.0   39286.29
10   11.0   43455.06
11   12.0   44893.31
